In [1]:
from langchain_core.documents import Document
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import TextLoader

In [3]:
loader=TextLoader("./knowledge_base/dff.txt",encoding="utf-8")
document=loader.load()
print(document)

[Document(metadata={'source': './knowledge_base/dff.txt'}, page_content="[Keyword]: Dff\n\n[Design Category]: Sequential Logic\n\n[Design Function Description]:\nThis design is a simple D flip-flop. It captures the value of the input signal 'd' on the rising edge of the clock signal 'clk' and outputs it as 'q'. The output 'q' retains its value until the next rising edge of 'clk'.\n\n[Input Signal Description]:\nclk: Clock signal used to synchronize the data capture. The flip-flop captures the input 'd' on the rising edge of this clock.\nd: Data input signal that is captured by the flip-flop on the rising edge of the clock.\n\n[Output Signal Description]:\nq: The output signal that holds the value of the input 'd' after the rising edge of the clock. It retains this value until the next rising edge of the clock.\n\n[Design Detail]: \nmodule topmodule (\n    input clk,    // Clocks are used in sequential circuits\n    input d,\n    output reg q );//\n\n    always @(posedge clk) begin\n   

In [4]:
### Directory Loader
from langchain_community.document_loaders import DirectoryLoader

## load all the text files from the directory
dir_loader=DirectoryLoader(
    "./knowledge_base",
    glob="**/*.txt", ## Pattern to match files  
    loader_cls= TextLoader, ##loader class to use
    loader_kwargs={'encoding': 'utf-8'},
    show_progress=False

)

documents=dir_loader.load()
documents

[Document(metadata={'source': 'knowledge_base\\dff.txt'}, page_content="[Keyword]: Dff\n\n[Design Category]: Sequential Logic\n\n[Design Function Description]:\nThis design is a simple D flip-flop. It captures the value of the input signal 'd' on the rising edge of the clock signal 'clk' and outputs it as 'q'. The output 'q' retains its value until the next rising edge of 'clk'.\n\n[Input Signal Description]:\nclk: Clock signal used to synchronize the data capture. The flip-flop captures the input 'd' on the rising edge of this clock.\nd: Data input signal that is captured by the flip-flop on the rising edge of the clock.\n\n[Output Signal Description]:\nq: The output signal that holds the value of the input 'd' after the rising edge of the clock. It retains this value until the next rising edge of the clock.\n\n[Design Detail]: \nmodule topmodule (\n    input clk,    // Clocks are used in sequential circuits\n    input d,\n    output reg q );//\n\n    always @(posedge clk) begin\n    

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

def split_documents(documents, chunk_size=500, chunk_overlap=50):
    if not documents:
        print("No documents to split.")
        return []

    # If all items are plain strings, convert them to Document objects
    if all(isinstance(d, str) for d in documents):
        documents = [Document(page_content=doc, metadata={}) for doc in documents]

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " "],
    )

    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    if split_docs:
        print("\nExample chunk:")
        print(f"Content : {split_docs[0].page_content[:200]}...")
        print(f"Metadata : {split_docs[0].metadata}")

    return split_docs

# --- New part for handling a .txt file ---

# 1. Load the text file using LangChain's TextLoader
# Make sure to replace 'your_file.txt' with the correct file path.
try:
    loader = TextLoader("./knowledge_base/dff.txt")
    documents = loader.load()
    
    # 2. Split the loaded documents
    chunks = split_documents(documents, chunk_size=500, chunk_overlap=50)
    print(f"Total chunks created: {len(chunks)}")
    
except FileNotFoundError:
    print("The file 'your_file.txt' was not found. Please check the file path.")


Split 1 documents into 3 chunks

Example chunk:
Content : [Keyword]: Dff

[Design Category]: Sequential Logic

[Design Function Description]:
This design is a simple D flip-flop. It captures the value of the input signal 'd' on the rising edge of the clock s...
Metadata : {'source': './knowledge_base/dff.txt'}
Total chunks created: 3


In [9]:
###Embedding and VectorStoreDB
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List,Dict,Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\shrut\OneDrive\Desktop\RAG_RTL\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    def __init__(self,model_name:str="all-MiniLM-L6-v2"):
        """
        Intialize the embedding manager
        Args:
           model_name:HuggingFace model name for sentence embedding
        """
        self.model_name = model_name
        self.model = None
        self._load_model()
    
    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model:{self.model_name}")
            self.model =SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}:{e}")
            raise
    
    def generate_embeddingd(self,texts:List[str])->np.ndarray:
        """AnyGenerate embeddings for a list of texts
        Args:
        texts:list of text strings to embed
        Returns:
        numpy array of embeddings with shape (len(texts),embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts,show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
embedding_manager= EmbeddingManager()
embedding_manager

Loading embedding model:all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


In [13]:
#VectorStore:
import os
class VectorStore:
     """ Manages document embedding in a ChromaDB vector store"""
     def __init__(self, collection_name:str ="txt_documents",persist_directory:str="../data/vector_store"):
            """
            Initialize the vector store
            Args:
            collection_name:name of the ChromaDB collection
            perist_directory: directory to persist the vector store
            """
            self.collection_name = collection_name
            self.persist_directory = persist_directory
            self.client = None
            self.collection = None
            self._initialize_store()

     def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client =chromadb.PersistentClient(path=self.persist_directory)
                
            self.collection =self.client.get_or_create_collection(
                    name=self.collection_name,
                    metadata={"description":"PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
                print(f"Error initializing vector store:{e}")
                raise

     def add_documents(self, documents: List[Any], embeddings: np.ndarray):
            """
            Add documents and their embeddings to the vector store
            Args:
            documents: list of LangChain documents
            embeddings: corresponding embedding for the documents
            """
            if len(documents)!= len(embeddings):
                raise ValueError("Number of documents must match number of embeddings")

            print(f"Adding {len(documents)} documents to vector store ...")

            ids = []
            metadatas = []
            documents_text = []
            embeddings_list =[]
            for i, (doc,embedding) in enumerate(zip(documents, embeddings)):
                #Generate unique ID
                doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
                ids.append(doc_id)
                
                #Prepare metadat
                metadata = dict(doc.metadata)
                metadata['doc index'] = i
                metadata['content_length'] = len(doc.page_content)
                metadatas.append(metadata)

                #Document context
                documents_text.append(doc.page_content)

                #Embedding
                embeddings_list.append(embedding.tolist())
            
            try: 
                self.collection.add(
                    ids = ids,
                    embeddings = embeddings_list,
                    metadatas = metadatas,
                    documents = documents_text
                )
                print(f"Successfully added {len(documents)} documents to vector store")
                print(f"Total documents in collection: {self.collection.count()}")
            except Exception as e:
                print(f"Error adding documents to vector store: {e}")
                raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: txt_documents
Existing documents in collection: 0


In [14]:
texts=[doc.page_content for doc in chunks]
texts

["[Keyword]: Dff\n\n[Design Category]: Sequential Logic\n\n[Design Function Description]:\nThis design is a simple D flip-flop. It captures the value of the input signal 'd' on the rising edge of the clock signal 'clk' and outputs it as 'q'. The output 'q' retains its value until the next rising edge of 'clk'.",
 "[Input Signal Description]:\nclk: Clock signal used to synchronize the data capture. The flip-flop captures the input 'd' on the rising edge of this clock.\nd: Data input signal that is captured by the flip-flop on the rising edge of the clock.\n\n[Output Signal Description]:\nq: The output signal that holds the value of the input 'd' after the rising edge of the clock. It retains this value until the next rising edge of the clock.",
 '[Design Detail]: \nmodule topmodule (\n    input clk,    // Clocks are used in sequential circuits\n    input d,\n    output reg q );//\n\n    always @(posedge clk) begin\n        q <= d;\n    end\n\nendmodule']

In [15]:
embeddings =embedding_manager.generate_embeddingd(texts)
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 3 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.13it/s]


Generated embeddings with shape: (3, 384)
Adding 3 documents to vector store ...
Successfully added 3 documents to vector store
Total documents in collection: 3


In [16]:
class RAGRetriever:
    """handles query-based retrieval from the vector store"""
    def __init__(self, vector_store: VectorStore,embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        Args:
           vector_store: Vector store containing document embeddings
           embedding_manager: manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query:str, top_k: int =5, score_threshold: float=0.0)-> List[Dict[str,Any]]:
        """
        Retrive relevant documents for a query
        Args:
        query: the search query
        top_k: Number of top results to return
        score_threshold: Minimum similarity score threshold

        Returns:
        List of dictionaries cantaining retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}")
        print(f"Top K:{top_k}, score threshold: {score_threshold}")

        #Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddingd([query])[0]
        
         #Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            #Process results
            retrieved_docs=[]
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids =results['ids'][0]

                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids,documents,metadatas,distances)):

                    similarity_score = 1-distance
                    if similarity_score>=score_threshold:
                        retrieved_docs.append({
                            'id':doc_id,
                            'content':document,
                            'metadata':metadata,
                            'similarity_score':similarity_score,
                            'distance':distance,
                            'rank':i+1
                        })
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            return retrieved_docs
        
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return[]
        
rag_retriever=RAGRetriever(vectorstore,embedding_manager)


In [17]:
rag_retriever
rag_retriever.retrieve("Inputs of a D flip flop?")

Retrieving documents for query: 'Inputs of a D flip flop?
Top K:5, score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 17.48it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_299b76d0_0',
  'content': "[Keyword]: Dff\n\n[Design Category]: Sequential Logic\n\n[Design Function Description]:\nThis design is a simple D flip-flop. It captures the value of the input signal 'd' on the rising edge of the clock signal 'clk' and outputs it as 'q'. The output 'q' retains its value until the next rising edge of 'clk'.",
  'metadata': {'doc index': 0,
   'content_length': 304,
   'source': './knowledge_base/dff.txt'},
  'similarity_score': 0.12181931734085083,
  'distance': 0.8781806826591492,
  'rank': 1},
 {'id': 'doc_bc9512d4_1',
  'content': "[Input Signal Description]:\nclk: Clock signal used to synchronize the data capture. The flip-flop captures the input 'd' on the rising edge of this clock.\nd: Data input signal that is captured by the flip-flop on the rising edge of the clock.\n\n[Output Signal Description]:\nq: The output signal that holds the value of the input 'd' after the rising edge of the clock. It retains this value until the next rising e

In [18]:
from langchain_community.chat_models import ChatOllama
import os
from dotenv import load_dotenv

load_dotenv()

# Replace the Groq model with your local Ollama-based Code Llama model
llm = ChatOllama(model="codellama", temperature=0.1)

# The rag_simple function remains the same as it's model-agnostic
def rag_simple(query, retriever, llm, top_k=3):
    results = retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""

    if not context:
        return "No relevant documents found to answer the query."
    
    prompt = f"""Use the following context to answer the question concisely.
    Context:
    {context}
    Question: {query}
    Answer:"""
    
    response = llm.invoke(prompt)  # No need for a list in Ollama
    return response.content

C:\Users\shrut\AppData\Local\Temp\ipykernel_30792\3135994308.py:8: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="codellama", temperature=0.1)


In [20]:
answer=rag_simple("Verilog Code for a D flip-flop",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'Verilog Code for a D flip-flop
Top K:3, score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 26.76it/s]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)


```
module d_ff(
  input clk,
  input d,
  output reg q
);
always @(posedge clk) begin
  if (d) begin
    q <= 1'b1;
  end else begin
    q <= 1'b0;
  end
end
endmodule
```


In [21]:
def rag_advanced(query, retriever, llm, top_k=200, min_score=0, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

In [22]:
result = rag_advanced("I would like you to implement a module named TopModule with the following interface. All input and output ports are one bit unless otherwise specified.- input clk, - input d, - output q. The module should implement a single D flip-flop. Assume all sequential logic is trigerred on the positive edge of the clock.", rag_retriever, llm,top_k=200,min_score=0,return_context=True)
print("Answer:", result['answer'])

Retrieving documents for query: 'I would like you to implement a module named TopModule with the following interface. All input and output ports are one bit unless otherwise specified.- input clk, - input d, - output q. The module should implement a single D flip-flop. Assume all sequential logic is trigerred on the positive edge of the clock.
Top K:200, score threshold: 0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 18.37it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


Answer: 
Here's an example implementation of the TopModule module with the specified interface:
```scss
module topmodule (
    input clk,
    input d,
    output reg q );

    always @(posedge clk) begin
        q <= d;
    end

endmodule
```
This implementation uses a single D flip-flop to capture the value of the input signal 'd' on the rising edge of the clock signal 'clk' and outputs it as 'q'. The output 'q' retains its value until the next rising edge of 'clk'.
